In [8]:
using Revise
using TORA

In [12]:
using BenchmarkTools
# using JLD2
using MeshCat
# using ProgressMeter
# using SparseArrays
# using StaticArrays
# using ThreadsX

In [13]:
using FastDifferentiation

In [14]:
if !@isdefined vis
    vis = Visualizer()
    robot_global = TORA.create_robot_franka("panda_arm", vis)
end

[ Info: Listening on: 127.0.0.1:8700, thread id: 1
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8700


Robot{Float64, RigidBodyDynamics.StateCache{Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{RigidBodyDynamics.Joint{Float64, RigidBodyDynamics.Revolute{Float64}}}}, 1}}, 7, 7, 7}("/Users/henrique/git/TORA.jl/src/../robots/panda_arm.urdf", Spanning tree:
Vertex: world (root)
  Vertex: panda_link1, Edge: panda_joint1
    Vertex: panda_link2, Edge: panda_joint2
      Vertex: panda_link3, Edge: panda_joint3
        Vertex: panda_link4, Edge: panda_joint4
          Vertex: panda_link5, Edge: panda_joint5
            Vertex: panda_link6, Edge: panda_joint6
              Vertex: panda_link7, Edge: panda_joint7
No non-tree joints., MechanismState{Float64, Float64, Float64, …}(…), StateCache{…}(…), DynamicsResultCache{Float64}(…), RigidBodyDynamics.SegmentedVectorCache{RigidBodyDynamics.JointID, Base.OneTo{RigidBodyDynamics.JointID}}(RigidBodyDynamics.CustomCollections.IndexDict{RigidBodyDynamics.JointID, Base.OneTo{RigidBodyDynamics.JointID}, UnitRange{Int64}}(RigidBodyDynamic

In [18]:
dt = 0.01

function prepare_jacdata()
    robot = TORA.create_robot_franka("panda_arm", vis)
    input = rand(robot.n_q + robot.n_v + robot.n_τ + robot.n_q + robot.n_v)
    output = rand(robot.n_q + robot.n_v)
    f!(defects, x) = TORA.forward_dynamics_defects!(defects, robot, x, dt)
    jacdata_fwd_dyn = TORA.JacobianData(f!, output, input)
    return jacdata_fwd_dyn
end

jacdata_fwd_dyn = prepare_jacdata();

## Benchmarking

In [19]:
let
    robot = TORA.create_robot_franka("panda_arm", vis)
    x = rand(robot.n_q + robot.n_v + robot.n_τ + robot.n_q + robot.n_v)
    defects = zeros(robot.n_q + robot.n_v)

    # Function evaluation
    display(@benchmark TORA.forward_dynamics_defects!($defects, $robot, $x, $dt))

    # Jacobian evaluation
    display(@benchmark jacdata_fwd_dyn($x))
end

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.637 μs …  13.783 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.679 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.731 μs ± 254.837 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▅██▆▃▃▂▁  ▂▂                                               ▂
  █████████▇████▇▇▆▅▅▆▄▅▆▆▆▆▆▅▅▅▅▄▅▆▅▅▅▆▄▆▅▆▆▆▆▆▅▆▆▆▅▆▇▇▇█▇▇▆ █
  1.64 μs      Histogram: log(frequency) by time      2.39 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  33.875 μs …   6.919 ms  ┊ GC (min … max): 0.00% … 99.16%
 Time  (median):     36.417 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.400 μs ± 109.150 μs  ┊ GC (mean ± σ):  4.76% ±  1.71%

       ▂▆█▃                                                     
  ▁▂▄▇▇████▆▄▃▂▃▃▄▄▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  33.9 μs         Histogram: frequency by time         54.2 μs <

 Memory estimate: 37.28 KiB, allocs estimate: 45.

In [21]:
let
    robot = TORA.create_robot_franka("panda_arm", vis)
    x = make_variables(:x, robot.n_q + robot.n_v + robot.n_τ + robot.n_q + robot.n_v)
    make_function([TORA.forward_dynamics_defects!], x)
end

LoadError: MethodError: no method matching make_function(::Vector{typeof(TORA.forward_dynamics_defects!)}, ::Vector{FastDifferentiation.Node})

[0mClosest candidates are:
[0m  make_function([91m::AbstractArray{T}[39m, ::AbstractVector{<:FastDifferentiation.Node}...; in_place) where T<:FastDifferentiation.Node
[0m[90m   @[39m [36mFastDifferentiation[39m [90m~/.julia/packages/FastDifferentiation/oGU1A/src/[39m[90m[4mCodeGeneration.jl:141[24m[39m
